In [ ]:
import fastai
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)

In [ ]:
path.ls()

In [ ]:
btfms = aug_transforms()+[Normalize.from_stats(*mnist_stats)]
btfms

In [ ]:
def image2image(x):
    return x

In [ ]:
block = DataBlock(blocks=(ImageBlock(cls=PILImageBW), ImageBlock(cls=PILImageBW)),
                  get_items = get_image_files,
                  get_y = image2image,
                  splitter=GrandparentSplitter(),
                  item_tfms=Resize(32),
                  batch_tfms = btfms,
)

In [ ]:
dls = block.dataloaders(path, batch_size=256)

In [ ]:
dls.show_batch()

In [ ]:
arch = create_body(xresnet18(pretrained=True), n_in=1).cuda()

In [ ]:
x,y = dls.one_batch()
x.shape, y.shape

In [ ]:
arch(x).shape

In [ ]:
class UpsampleBlock(Module):
    def __init__(self, up_in_c:int, final_div:bool=True, blur:bool=False, leaky:float=None, **kwargs):
        self.shuf = PixelShuffle_ICNR(up_in_c, up_in_c//2, blur=blur, **kwargs)
        ni = up_in_c//2
        nf = ni if final_div else ni//2
        self.conv1 = ConvLayer(ni, nf, **kwargs)
        self.conv2 = ConvLayer(nf, nf, **kwargs)
        self.relu = nn.ReLU()

    def forward(self, up_in:Tensor) -> Tensor:
        up_out = self.shuf(up_in)
        cat_x = self.relu(up_out)
        return self.conv2(self.conv1(cat_x))

In [ ]:
def decoder_resnet(y_range, n_out=1):
    return nn.Sequential(UpsampleBlock(512), 
                         UpsampleBlock(256),
                         UpsampleBlock(128),
                         UpsampleBlock(64),
                         UpsampleBlock(32),
                         nn.Conv2d(16, n_out, 1),
                         SigmoidRange(*y_range)
                        )
                         
def autoencoder(encoder, y_range): return nn.Sequential(encoder, decoder_resnet(y_range))

In [ ]:
y_range = (-3.,3.)
ac_resnet = autoencoder(arch, y_range).cuda()

In [ ]:
dec = decoder_resnet(y_range).cuda()

In [ ]:
dec(arch(x)).shape

In [ ]:
learn = Learner(dls, ac_resnet, loss_func=MSELossFlat())

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.show_results()

In [ ]:
learn.fit_one_cycle(10)

In [ ]:
learn.show_results()